<h1><center>CMPE 462 - Project 2<br>Implementing an SVM Classifier<br>Due: May 18, 2020, 23:59</center></h1>

* **Student ID1:**
* **Student ID2:**
* **Student ID3:**

## Overview

In this project, you are going to implement SVM. For this purpose, a data set (data.mat) is given to you. You can load the mat dataset into Python using the function `loadmat` in `Scipy.io`. When you load the data, you will obtain a dictionary object, where `X` stores the data matrix and `Y` stores the labels. You can use the first 150 samples for training and the rest for testing. In this project, you will use the software package [`LIBSVM`](http://www.csie.ntu.edu.tw/~cjlin/libsvm/) to implement SVM. Note that `LIBSVM` has a [`Python interface`](https://github.com/cjlin1/libsvm/tree/master/python), so you can call the SVM functions in Python. 

In [1]:
!pip3 install libsvm
import scipy.io as spio
from libsvm.svmutil import *

dic = spio.loadmat("data.mat")

train_matrix, train_labels = dic['X'][:150], dic['Y'][:150]
test_matrix, test_labels = dic['X'][150:], dic['Y'][150:]

train_labels = train_labels.reshape((train_labels.shape[0],))
test_labels = test_labels.reshape((test_labels.shape[0],))


## Task 1 - 30 pts

Train a hard margin linear SVM and report both train and test classification accuracy.

In [9]:
m = svm_train(train_labels,train_matrix,'-t 0 -c 100000000')
p_label, p_acc, p_val = svm_predict(train_labels, train_matrix, m)
print("Train classification accuracy: " + str(p_acc[0]))

Accuracy = 74.6667% (112/150) (classification)
Train classification accuracy: 74.66666666666667


In [10]:
p_label, p_acc, p_val = svm_predict(test_labels, test_matrix, m)
print("Test classification accuracy: " + str(p_acc[0]))

Accuracy = 77.5% (93/120) (classification)
Test classification accuracy: 77.5


## Task 2 - 40 pts

Train soft margin SVM for different values of the parameter $C$, and with different kernel functions. Systematically report your results. For instance, report the performances of different kernels for a fixed $C$, then report the performance for different $C$ values for a fixed kernel, and so on.

## Task 3 - 15 pts

Please report how the number of support vectors changes as the value of $C$ increases (while all other parameters remain the same). Discuss whether your observations match the theory.

## Task 4 - 15 pts

Please investigate the changes in the hyperplane when you remove one of the support vectors, vs., one data point that is not a support vector.

### Bonus Task - 10 pts

Use Python and [CVXOPT](http://cvxopt.org) QP solver to implement the hard margin SVM. 